In [5]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
!unzip -q kagglecatsanddogs_3367a.zip

--2021-11-09 09:25:29--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.200.148.143, 2600:140e:6:98d::e59, 2600:140e:6:9be::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.200.148.143|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip.1’

kagglecatsanddogs_3 100%[===================>] 786.68M   142MB/s    in 5.7s    

2021-11-09 09:25:35 (138 MB/s) - ‘kagglecatsanddogs_3367a.zip.1’ saved [824894548/824894548]

replace MSR-LA - 3467.docx? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace readme[1].txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
import os
import numpy as np
import shutil
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

cat_files = os.listdir('PetImages/Cat')
dog_files = os.listdir('PetImages/Dog')

for cat in cat_files:
  src = os.path.join('PetImages/Cat',cat)
  dst = os.path.join('PetImages/Cat','cat_'+cat)
  os.rename( src,dst )

for dog in dog_files:
  src = os.path.join('PetImages/Dog',dog)
  dst = os.path.join('PetImages/Dog','dog_'+dog)
  os.rename( src , dst )
  

cat_files = tf.io.gfile.glob('PetImages/Cat/*')
dog_files = tf.io.gfile.glob('PetImages/Dog/*')

print(len(cat_files),len(dog_files))

cat_train = np.random.choice(cat_files, size=3000, replace=False)
dog_train = np.random.choice(dog_files, size=3000, replace=False)
cat_files = list(set(cat_files) - set(cat_train))
dog_files = list(set(dog_files) - set(dog_train))

cat_val = np.random.choice(cat_files, size=1000, replace=False)
dog_val = np.random.choice(dog_files, size=1000, replace=False)
cat_files = list(set(cat_files) - set(cat_val))
dog_files = list(set(dog_files) - set(dog_val))

cat_test = np.random.choice(cat_files, size=1000, replace=False)
dog_test = np.random.choice(dog_files, size=1000, replace=False)

print('Cat datasets:', cat_train.shape, cat_val.shape, cat_test.shape)
print('Dog datasets:', dog_train.shape, dog_val.shape, dog_test.shape)

18802 18789
Cat datasets: (3000,) (1000,) (1000,)
Dog datasets: (3000,) (1000,) (1000,)


In [7]:
train_dir = 'training_data'
val_dir = 'validation_data'
test_dir = 'test_data'

train_files = np.concatenate([cat_train, dog_train])
validate_files = np.concatenate([cat_val, dog_val])
test_files = np.concatenate([cat_test, dog_test])

os.mkdir(train_dir) if not os.path.isdir(train_dir) else None
os.mkdir(val_dir) if not os.path.isdir(val_dir) else None
os.mkdir(test_dir) if not os.path.isdir(test_dir) else None

for fn in train_files:
    shutil.copy(fn, train_dir)

for fn in validate_files:
    shutil.copy(fn, val_dir)
    
for fn in test_files:
    shutil.copy(fn, test_dir)

In [8]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

IMG_DIM = (150,150)


train_files = tf.io.gfile.glob('training_data/*')
train_imgs = [];train_labels = []
for file in train_files:
  try:
    train_imgs.append( img_to_array(load_img( file,target_size=IMG_DIM )) )
    train_labels.append(file.split('/')[1].split('_')[0])
  except:
    pass
train_imgs = np.array(train_imgs)

validation_files = tf.io.gfile.glob('validation_data/*')
validation_imgs = [];validation_labels = []
for file in validation_files:
  try:
    validation_imgs.append( img_to_array(load_img( file,target_size=IMG_DIM )) )
    validation_labels.append(file.split('/')[1].split('_')[0])
  except:
    pass
train_imgs = np.array(train_imgs)
validation_imgs = np.array(validation_imgs)


print('Train dataset shape:', train_imgs.shape, 
      '\tValidation dataset shape:', validation_imgs.shape)

Train dataset shape: (11506, 150, 150, 3) 	Validation dataset shape: (3944, 150, 150, 3)


In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)
validation_labels_enc = le.transform(validation_labels)

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   zoom_range=0.3, 
                                   rotation_range=50,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow(train_imgs, train_labels_enc, batch_size=30)
val_generator = val_datagen.flow(validation_imgs, validation_labels_enc, batch_size=20)

In [11]:

from keras.layers import  Flatten, Dense, Dropout

from keras.models import Model
from keras import optimizers

input_shape = (150, 150, 3)
vgg = tf.keras.applications.vgg16.VGG16(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=input_shape, pooling=None
)




vgg.trainable = False
for layer in vgg.layers[:-8]:
    layer.trainable = False

vgg_output = vgg.layers[-1].output

fc1 = Flatten()(vgg_output)
fc1 = Dense(512, activation='relu')(fc1)
fc1_dropout = Dropout(0.3)(fc1)

fc2 = Dense(512, activation='relu')(fc1_dropout)
fc2_dropout = Dropout(0.3)(fc2)

output = Dense(1, activation='sigmoid')(fc2_dropout)
model = Model(vgg.input, output)


model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])


model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [12]:
import pandas as pd
layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at...,input_1,False
1,<keras.layers.convolutional.Conv2D object at 0...,block1_conv1,False
2,<keras.layers.convolutional.Conv2D object at 0...,block1_conv2,False
3,<keras.layers.pooling.MaxPooling2D object at 0...,block1_pool,False
4,<keras.layers.convolutional.Conv2D object at 0...,block2_conv1,False
5,<keras.layers.convolutional.Conv2D object at 0...,block2_conv2,False
6,<keras.layers.pooling.MaxPooling2D object at 0...,block2_pool,False
7,<keras.layers.convolutional.Conv2D object at 0...,block3_conv1,False
8,<keras.layers.convolutional.Conv2D object at 0...,block3_conv2,False
9,<keras.layers.convolutional.Conv2D object at 0...,block3_conv3,False


In [ ]:
from keras.callbacks import EarlyStopping

filepath="saved_models/vgg_transfer_learn_dogvscat.h5"

save_model_cb = ModelCheckpoint(filepath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')
# callback to stop the training if no improvement
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=7, mode='min')

callbacks_list = [save_model_cb,early_stopping_cb]

history = tf.keras.Model(train_generator, steps_per_epoch=100, epochs=100,
                              validation_data=val_generator, validation_steps=50, 
                              verbose=2,callbacks=callbacks_list)

In [ ]:
import matplotlib.pyplot as plt

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('Basic CNN Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)


epoch_list = history.epoch
ax1.plot(epoch_list, history.history['acc'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_acc'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, epoch_list[-1], 3))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, epoch_list[-1], 3))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")